# Deep Learning With Stroke Data

# İnme nedir?
Beyninizin bir kısmına giden kan akışı azaldığında veya kesintiye uğradığında, beyin dokusunun oksijen ve besin almasını önleyen bir inme (felç) meydana gelir ve beyin hücreleri dakikalar içerisinde ölmeye başlar. İnmenin oluştuğu bölgedeki fonksiyonlarda geçici ya da kalıcı kayıplar oluşur.

# About Stroke
Stroke is a disease that affects the arteries leading to and within the brain. A stroke occurs when a blood vessel that carries oxygen and nutrients to the brain is either blocked by a clot or bursts . When that happens, part of the brain cannot get the blood it needs, so it and brain cells die.

Elimizde inme geçirmiş ve geçirmemiş kişilerin;
* Cinsiyeti
* Yaşı
* Hipertansiyonu olup olmadığı
* Kalp hastalığı geçirip geçirmediği
* Evlenip evlenmediği
* Çalışıp çalışmadığı
* İkamet ettiği yerin tipi
* Kanındaki ortalama glukoz değeri
* Vücut kütle indeksi

verileri bulunmakta. Biz de bu veriler ile bir derin öğrenme modeli uygulayarak, verileri girdi olarak verdiğimizde kişinin inme geçirip geçirmeyeceğini bize söyleyen bir yapay zeka programlayacağız.

We have the information about
* Gender
* Age
* Hypertension
* Heart Disease
* Marriage
* Work type
* Residance type
* Avarage glucose level
* Body mass index
* Smoking status
of a person that have had a stroke or not. We are going to use this data and make a deep learning model that can predict the stroke status with given information.

Kullanacağımız kütüphaneleri ekleyelim.

Let's add the libraries

In [ ]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

*Verilerimizi kaydedelim* 

Now save the dataset to a variable that named "st"

In [ ]:
st=pd.read_csv('../input/stroke-prediction-dataset/healthcare-dataset-stroke-data.csv')
st

In [ ]:
graphst = st

In [ ]:
st.insert(0, 'ID', range(1, 1 + len(st)))

Male = 0 - Female = 1,
Ever_married, Yes = 1 - No = 0,
Work_type, Private = 0.050940 - Self-employed = 0.079365 - Govt_job = 0.050228 - children = 0.002911 - Never_worked = 0,
Residence_type, Urban = 1 - Rural = 0,
Smoking_status, Formarly smoked = 0.079096 - smokes = 0.053232 - never smoked = 0.047569, unknown = 0.030440,
bmi, NaN = 28.893236911794673

Şimdi verimizi düzenleyelim. Modeli uygulamak için yazılı olan değerleri numerik değerlere çevirdim. Yukarıda çevirdiğim değerleri ve yeni değerlerini bulabilirsiniz. Evet, hayır gibi net durumlara 0 ve 1 değerlerini kullandım, onun dışında daha komplike durumlara ise inmeye etki ettiği ortalama değerleri verdim. Aynı zamanda cinsiyet için "Other" verisinin bulunduğu girdiyi sildim tek bir girdi olduğu için.

Not:
* 249 adet inme geçirmiş kişinin verisi bulunmakta.

As you can see we have string values in the dataset, we should change this string values into numeric ones and there is a column named "id" that we don't need to use so I will drop that column and change the values. In the "gender" column we have a data named "other" and I will delete that because it is the only data named "other" and deleting it doesn't make a difference.

You can see the new values of the strings below the dataset. You may have a question about work type values and smoking status values. Let me explain this.
I used 0,1 numbers for the "yes or no" type questions and for the more complicated ones like "smoking status" I used the the numeric values that are the mean value of the stroke status effect of classes.

Note that we have 249 datas with people who have had stroke and they are in the first 249 lines of the dataset.

In [ ]:
st.drop("id",axis=1,inplace=True)


In [ ]:
st = st.replace("Male", 0)
st = st.replace("Female", 1)
st = st.replace("Yes", 1)
st = st.replace("No", 0)
st = st.replace("Private", 0.050940)
st = st.replace("Self-employed", 0.079365)
st = st.replace("Govt_job", 0.050228)
st = st.replace("children", 0.002911)
st = st.replace("Never_worked", 0)
st = st.replace("Urban", 1)
st = st.replace("Rural", 0)
st = st.replace("smokes", 0.053232)
st = st.replace("Unknown", 0.030440)
st = st.replace("never smoked", 0.047569)
st = st.replace("formerly smoked", 0.079096)
st = st.replace(np.nan,28.893236911794673 )

In [ ]:
index = st.index
x = st["gender"] == "Other"
index[x]
st = st.drop(index[x])

In [ ]:
st.reset_index(drop=True, inplace=True)

Şimdi verilerimizi bir kaç grafik yardımıyla inceleyelim.

Now Let's examine our dataset with the help of some graphics.

In [ ]:
plt.figure(figsize=(12,2))
plt.plot(st[:500].ID,st[:500].avg_glucose_level)
plt.title("İlk 500 verinin ortalama glukoz değeri / Avarage glucose level of first 500 datas")
plt.xlabel("Sıra, Ids")
plt.ylabel("Ortalama glukoz değeri, Avarage glucose level")
plt.show()

plt.figure(figsize=(12,2))
plt.plot(st[:500].ID,st[:500].age)
plt.title("İlk 500 verinin yaşı / Age of first 500 datas")
plt.xlabel("Sıra, Ids")
plt.ylabel("Yaş, Age")
plt.show()

plt.figure(figsize=(12,2))
plt.plot(st[:500].ID,st[:500].bmi)
plt.title("İlk 500 verinin vücut kitle indeksi / Body Mass İndex of first 500 datas")
plt.xlabel("Sıra, Ids")
plt.ylabel("Vücut kitle indeksi, Body Mass İndex")
plt.show()

In [ ]:
l = graphst[:249]
x = l.ID
y = l.work_type
plt.figure(figsize=(12,2))
plt.title("""Work type of "Have had stroke" datas / İnme geçirenlerin çalışma durumu""")
plt.scatter(x, y, color = "red")
plt.show()

x = l.ID
y = l.Residence_type
plt.figure(figsize=(12,2))
plt.title("""Residance type of "Have had stroke" datas / İnme geçirenlerin ikamet durumu""")
plt.scatter(x, y, color = "orange")
plt.show()

x = l.ID
y = l.smoking_status
plt.figure(figsize=(12,2))
plt.title("""Smoking status of "Have had stroke" datas / İnme geçirenlerin sigara içme durumu""")
plt.scatter(x, y, color = "black")
plt.show()

In [ ]:
l=st[:249]
a = st["stroke"].where(st["stroke"]==1)
a=a.dropna()
b = st["stroke"].where(st["stroke"]==0)
b=b.dropna()
labels = 'Have Had Stroke / inme geçirmiş', 'Have Not Had Stroke / inme geçirmemiş'
sizes = [a.size, b.size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("Stroke status of all datas / İnme geçirme durumu")
plt.show()

a= l["hypertension"].where(st["hypertension"]==1)
b= l["hypertension"].where(st["hypertension"]==0)
labels = "Have hypertension / Hipertansiyonu var", "Don't have hypertension / Hipertansiyonu yok"
a=a.dropna()
b=b.dropna()
sizes = [a.size,b.size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("""Hypertension status of "Have had stroke" datas / İnme geçirenlerin hipertansiyon durumu""")
plt.show()

a= l["heart_disease"].where(st["heart_disease"]==1)
b= l["heart_disease"].where(st["heart_disease"]==0)
labels = "Have heart disease / Kalp hastalığı var", "Don't have heart disease / Kalp hastalığı yok"
a=a.dropna()
b=b.dropna()
sizes = [a.size,b.size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("""heart disease status of "Have had stroke" datas / İnme geçirenlerin kalp hastalığı durumu""")
plt.show()

a= l["ever_married"].where(st["ever_married"]==1)
b= l["ever_married"].where(st["ever_married"]==0)
labels = "Have ever married / Evlenmiş", "Don't have ever married / Evlenmemiş "
a=a.dropna()
b=b.dropna()
sizes = [a.size,b.size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("""ever married status of "Have had stroke" datas / İnme geçirenlerin evlilik durumu""")
plt.show()

a= l["gender"].where(st["gender"]==1)
b= l["gender"].where(st["gender"]==0)
labels = "Female / Kadın", "Male / Erkek"
a=a.dropna()
b=b.dropna()
sizes = [a.size,b.size]
fig1, ax1 = plt.subplots()
ax1.pie(sizes, labels=labels, autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal')
plt.title("""gender status of "Have had stroke" datas / İnme geçirenlerin cinsiyetleri""")
plt.show()

In [ ]:
st.drop("ID",axis=1,inplace=True)

Verilerimizin son hali aşağıdadır.

You can see the updated version of the dataset below

In [ ]:
st

Verilerimizin değerleri son durumda numerik olduğu için hepsinin tipini "float32" olarak değiştiriyorum.

I am changing dtypes to float32. 

In [ ]:
st.astype("float32").dtypes

Elimizde 249 adet inme geçirmiş kişinin verisi olduğu için ve bu veriler ilk sıradan 249.'ya kadar gittiğinden dolayı, ilk 900 veriyi modelde yürütmek için alıyorum. Bütün verileri koymamamın nedeni 4750 adet inme geçirmemiş verinin 249 inme geçiren verinin yanında modeli şaşırtabilme ihtimalini ortadan kaldırmak.

Now let's take the first 900 lines of the dataset. That means 249 "stroke status 1" datas and 651 "stroke status 0" datas.  

In [ ]:
st = st[:900]

İnme geçirip geçirmeme durumunun olduğu sütunu kesiyorum, daha sonra bu sütunu etiket olarak kullanacağız.

I am slicing the "stroke" column and save it in a variable named "stroke". Later we will use this variable as a label of inputs.

In [ ]:
stroke = st.pop('stroke')
stroke

X değerine inme durumu dışındaki verilerimizin değerlerini, Y değerine ise inme etiketlerimizi koyuyorum.

I am saving the values of the all lines in st(as I mentioned above all values except stroke status because we sliced it) to new variable "X" as a list format and I am saving the values of stroke to new variable "Y". As you can understand any number in Y is the label of the value list in X in the same row.

In [ ]:
X = st.values
Y = stroke.values

In [ ]:
X

Tekrar veri tipini "float32"'ye çeviriyorum.

Now we should change the dtypes of the X and Y to float32.

In [ ]:
X=np.asarray(X).astype(np.float32)
Y=np.asarray(Y).astype(np.float32)

Verilerimi "train" ve "test" olarak yüzde 80 ve yüzde 20'lik kısımlara ayırıyorum, modeli hazırlıyorum. Modelde yürüteceğimiz veri "train" kısmına ayırdığımız veri olacak, "test" kısmının verisi ile de modeli test edeceğiz.

I am seperating the X and Y's to train and test sets. We are going to use the train sets for model and test sets for prediction.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingClassifier
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20)

Testte optimum sonuç için farklı parametreler ile denemeler yapıyorum.

At this stage I am training our model with different parameters to find the optimum accuracy result.

In [ ]:
clf = GradientBoostingClassifier(n_estimators=5000, learning_rate=0.01,
    max_depth=1, random_state=10).fit(X_train, y_train)
print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

Modelimiz "train" setinde yüzde 85 doğruluk oranı verdi, bu modeli "test" setinde denediğimizde de yüzde 84 doğruluk oranı aldık. 5 verinin 4'ünü doğru sınıflandırıyor demektir kaba bir tabirle. Yüksek bir doğruluk oranına ulaştık.

Finally I got the %85 accuracy on train set and %84 accuracy on test set. These results will chance every run because of the seperate process we did before but avarage values are 84 and 83. This means we have reached a high accuracy rate.

Şimdi de "test" setindeki tahminleri bizim etiketlerimiz ile karşılaştıralım.

Now let's check our test results and test labels.

In [ ]:
clf.predict(X_test)[:140]==y_test[:140]

140 verinin 118 tanesini doğru sınıflandırmış, 22 tanesinde ise yanılmış.

The model predicted 118 inputs out of 140 true and made a wrong prediction on 22 inputs.